In [1]:
import pandas as pd
from rapidfuzz import process, fuzz
import src as src
pd.set_option('display.max_columns', None)

In [2]:
df_eventos = pd.read_csv("data/df_eventos_liga2015.csv")

C:\Users\pepel\AppData\Local\Temp\ipykernel_69628\3719518088.py:1: DtypeWarning: Columns (0,1,3,11,12,19,34,37,39,40,41,42,43,44,45,46,47,49,50,51,52,53,55,56,57,58,59,60,71,73,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116) have mixed types. Specify dtype option on import or set low_memory=False.
  df_eventos = pd.read_csv("data/df_eventos_liga2015.csv")


In [3]:
df_jugadores_info = pd.read_csv("data/jugadores_info.csv")

In [ ]:
df_jugadoresunicos_eventos = df_eventos["player"].unique().tolist()
df_jugadoresunicos_info = df_jugadores_info["playerFullName"].unique().tolist()
df_clubesunicos_eventos = df_eventos["team"].unique().tolist()
df_clubesunicos_info = df_jugadores_info["club"].unique().tolist()

In [ ]:
df_nombres = src.normalizacion_nombres(df_jugadoresunicos_info, df_jugadoresunicos_eventos)

In [ ]:
df_clubes = src.normalizacion_nombres(df_clubesunicos_info, df_clubesunicos_eventos)

In [ ]:
# Crear diccionario para mapear los nombres
dict_nombres_info = df_nombres.set_index('Nombre_original')['Nombre_unificado'].to_dict()
# Mapear nombres jugadores
df_jugadores_info["playerFullName"] = df_jugadores_info["playerFullName"].apply(
    lambda x: dict_nombres_info.get(x, x) if dict_nombres_info.get(x) is not None else x)

In [59]:
# Crear diccionario para mapear los equipos
df_equipos_jugadores = df_eventos[["player", "team"]].drop_duplicates()
dict_equipos_jugadores = dict(zip(df_equipos_jugadores["player"], df_equipos_jugadores["team"]))
# Mapear los equipos en la temporada 2015/2016 ya que la info que aparece es actual
df_jugadores_info["club"] = df_jugadores_info["playerFullName"].map(dict_equipos_jugadores)

In [62]:
# Crear diccionarios para mapear los IDs de jugador y equipo
df_equipos = df_eventos[["team", "team_id"]].drop_duplicates()
dict_equipos = dict(zip(df_equipos["team"], df_equipos["team_id"]))
df_jugadores = df_eventos[["player", "player_id"]].drop_duplicates()
dict_jugadores = dict(zip(df_jugadores["player"], df_jugadores["player_id"]))

In [63]:
# Mapear ID Jugadores
df_jugadores_info["playerID"] = df_jugadores_info["playerFullName"].map(dict_jugadores)
# Mapear ID Equipos
df_jugadores_info["clubID"] = df_jugadores_info["club"].map(dict_equipos)

In [65]:
df_jugadores_info.to_csv("data/jugadores_info_normalizado.csv", index=False)